## 1. Setup

In [1]:
import sys
sys.path.append('../..')

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import shutil
import warnings

from keras import Model
from neural_networks.fcrn import FCRN_A
from utils.data.data_generator import DataGenerator
from utils.data.data_ops import move_val_split_to_train
from utils.data.data_ops import create_val_split_from_train
from utils.input_output.io import save_np_arrays
from pprint import pprint

Using TensorFlow backend.


In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

warnings.filterwarnings('ignore')

## 2. Experiment parameters

In [4]:
DATASET_NAME = 'shanghai_tech/part_b'
DATASET_PATH = f'../../datasets/{DATASET_NAME}'
IMG_DIM = None
D_MAP_MULT_FACT = None
LOSS_FUNCTIONS = ['mse', 'mae', 'logcosh']
EXP_DIR_NAMES = []

if DATASET_NAME.lower() == 'vgg_cells':
    # misc
    IMG_DIM = (256, 256, 3)
    D_MAP_MULT_FACT = 100.
    
    # plots
    figsize=(25, 5)
    fraction = 0.045
    pad = 0.038
    
    # checkpoints
    for loss_name in LOSS_FUNCTIONS:
        for input_type in ['full_img', 'patch_32_128x128']:
            for train_size in [32, 64]:
                for rseed_idx in range(1, 6):
                    exp_dir_name =  f'./vgg_cells/'\
                                    f'n_{train_size}_sigma_5_'\
                                    f'randseed_{train_size}{rseed_idx}_'\
                                    f'loss_{loss_name}_'\
                                    f'{input_type}'
                    EXP_DIR_NAMES.append(exp_dir_name)
    
elif DATASET_NAME.lower() == 'carpk':
    # misc
    IMG_DIM = (720, 1280, 3)
    D_MAP_MULT_FACT = 2000.
    
    # plots
    figsize=(25, 7)
    fraction = 0.027
    pad = 0.015
    
    # checkpoints
    EXP_DIR_NAMES = ['./carpk/sigma_10_loss_logcosh_patch_32_128x128_15_epochs']
                    #['./carpk/sigma_10_loss_logcosh_patch_32_128x128_5_epochs']
    
elif DATASET_NAME.lower() == 'shanghai_tech/part_b':
    # misc
    IMG_DIM = (768, 1024, 3)
    D_MAP_MULT_FACT = 2000.
    
    # plots
    figsize=(25, 7)
    fraction = 0.0355
    pad = 0.015
    
    # checkpoints
    EXP_DIR_NAMES = ['./shanghai_tech/part_b/sigma_10_loss_logcosh_full_img_epochs_30']

In [5]:
params = {
    'dim': IMG_DIM,
    'batch_size': 1,
    'patches_per_image': 1,
    'density_map_multiplication_factor': D_MAP_MULT_FACT,
    'shuffle': False
}

In [6]:
print(len(EXP_DIR_NAMES))
pprint(EXP_DIR_NAMES)

1
['./shanghai_tech/part_b/sigma_10_loss_logcosh_full_img_epochs_30']


## 3. Predict and save results

In [7]:
for exp_dir_name in EXP_DIR_NAMES[:1]:
    if 'vgg_cells' in DATASET_PATH:
        # for vgg_cells dataset we divide the train set in different train/val split each time
        train_path = f'{DATASET_PATH}/train'
        val_path = f'{DATASET_PATH}/val'
        
        train_size = int(re.search(r'/n_\d{2}_', exp_dir_name).group().split('_')[1])
        val_size = 100 - train_size
        rand_seed = int(re.search(r'randseed_\d{3}_', exp_dir_name).group().split('_')[1])
        
        move_val_split_to_train(val_path, train_path)
        create_val_split_from_train(train_path, val_path, val_size, rand_seed)
        
    # data splits
    train_generator = DataGenerator(DATASET_PATH, 'train', **params)
    val_generator = DataGenerator(DATASET_PATH, 'val', **params)
    test_generator = DataGenerator(DATASET_PATH, 'test', **params)
    
    # clean old dir for qualitative results
    results_path = f'{exp_dir_name}/results/qualitative'
    shutil.rmtree(results_path, ignore_errors=True)
    os.makedirs(results_path)

    # predict and save
    checkpoint_filename = f'{exp_dir_name}/checkpoints/best_model.hdf5'
    model = FCRN_A(pretrained_weights=checkpoint_filename)
    
    for data_generator, split_name in zip([train_generator, val_generator, test_generator],
                                          ['train', 'val', 'test']):
        # prepare dirs for qualitative results
        results_path_npy = f'{results_path}/{split_name}/npy'
        results_path_png = f'{results_path}/{split_name}/png'
        results_path_png_diff = f'{results_path}/{split_name}/png_diff'
        results_path_png_gt_asc = f'{results_path}/{split_name}/png_gt_asc'
        os.makedirs(results_path_npy)
        os.makedirs(results_path_png)
        os.makedirs(results_path_png_diff)
        os.makedirs(results_path_png_gt_asc)
        
        for idx in range(data_generator.__len__()):
            # input image & gt density map, batch_size=1
            input_img, gt_density_map = data_generator.__getitem__(idx)
            gt_density_map /= D_MAP_MULT_FACT
            
            # predicted density map
            pred_density_map = (model.predict(input_img) / D_MAP_MULT_FACT)[0]
            
            # gt & pred counts
            gt_count = int(np.round(gt_density_map.sum()))
            pred_count = pred_density_map.sum()
            pred_gt_diff = pred_count - gt_count
            
            error_type = 'match'
            if pred_count < gt_count:
                error_type = 'underestimate'
            elif pred_count > gt_count:
                error_type = 'overestimate'
            
            img_name_png = data_generator.img_names[data_generator.indexes[idx]]
            img_name = img_name_png.split('.')[0]
            img_name_npy = f'{img_name}_gt_{gt_count}_pred_{pred_count:.2f}'\
                           f'_diff_gt_pred_{pred_gt_diff:.2f}'
            img_name_diff = f'{error_type}_{np.abs(pred_gt_diff):7.2f}'\
                            f'_gt_{gt_count}_pred_{pred_count:.2f}'\
                            f'_{img_name}'.replace(' ', '0')
            img_name_gt_asc = f'gt_{gt_count:3}_pred_{pred_count:6.2f}_diff_gt_pred_{pred_gt_diff:.2f}'\
                              f'_{img_name}'.replace(' ', '0')
            
            # .npy arrays
            save_np_arrays([pred_density_map], [img_name_npy], results_path_npy)
            
            # plots
            vmin = min(gt_density_map.min(), pred_density_map.min())
            vmax = max(gt_density_map.max(), pred_density_map.max())
            
            fig = plt.figure(figsize=figsize)

            plt.subplot(1, 4, 1)
            plt.title('Input image')
            plt.imshow(input_img[0])
            plt.axis('off')

            plt.subplot(1, 4, 2)
            plt.title(f'GT density map ({gt_count})')
            plt.imshow(gt_density_map.squeeze(), cmap='jet', vmin=vmin, vmax=vmax)
            plt.colorbar(fraction=fraction, pad=pad)
            plt.axis('off')

            plt.subplot(1, 4, 3)
            plt.title(f'Predicted density map ({pred_count:.2f})')
            plt.imshow(pred_density_map.squeeze(), cmap='jet', vmin=vmin, vmax=vmax)
            plt.colorbar(fraction=fraction, pad=pad)
            plt.axis('off')

            plt.subplot(1, 4, 4)
            plt.title(f'Prediction - GT ({(pred_density_map - gt_density_map).sum():.2f})')
            plt.imshow((pred_density_map - gt_density_map).squeeze(), cmap='gray')
            plt.colorbar(fraction=fraction, pad=pad)
            plt.axis('off')

            plt.savefig(f'{results_path_png}/{img_name_npy}.png')
            plt.savefig(f'{results_path_png_diff}/{img_name_diff}.png')
            plt.savefig(f'{results_path_png_gt_asc}/{img_name_gt_asc}.png')
            plt.clf()
            plt.close()